In [8]:
import pandas as pd
import numpy as np
import os

In [13]:
os.chdir("/home/edgan/projects/sketchstore/python")
DATA_DIR = "/mnt/disks/data/datasets/"

In [10]:
import testdata.bench_gen

In [11]:
def sample_until_lazy(xs, size, seed=0):
    r = np.random.RandomState(seed)
    i = 0
    res = set()
    while len(res) < size and len(xs) > 0:
        new_x = r.choice(xs)
        res.add(new_x)
        i += 1
        if i % 1000 == 0:
            xs = xs[np.isin(xs, res, invert=True)]
    return list(res)

In [12]:
df, dim_names = testdata.bench_gen.gen_data(
    10_000_000,
    [(10, 1),
     (10, 1),
     (10, 1),
     (10, 1),
     ],
    f_skew=1.1,
    f_card=50000,
    seed=0,
)

In [19]:
fname = os.path.join(DATA_DIR, "sketchstore_synth/bcube4_10M.feather")
tfname = os.path.join(DATA_DIR, "sketchstore_synth/bcube4_10M_f_track.csv")
tqname = os.path.join(DATA_DIR, "sketchstore_synth/bcube4_10M_q_track.csv")

In [ ]:
df.to_feather(fname)

In [17]:
df = pd.read_feather(fname)

In [20]:
x_track = sample_until_lazy(df["f"], size=200, seed=0)
df_track = pd.DataFrame({
    "x_track": x_track,
})
df_track.to_csv(
    tfname, 
    index=False,
)

In [21]:
x_track = np.percentile(df["q"], np.arange(0,101))
df_track = pd.DataFrame({
    "x_track": x_track,
})
df_track.to_csv(
    tqname, 
    index=False,
)